# Semantic Segmentation 自動ラベリング検証

このノートブックは、ラベリング済みのデータセットを使って自動ラベリングによってどの程度工数が削減できるかを検証するためのサンプルノートブックです。検証手順は以下の通りです。

1. Amazon SageMaker のビルトインアルゴリズムを使って、用意したラベルありデータセットから任意の数を取り出しモデルの学習を実施
1. 作成したモデルを使って、残りのデータ（未ラベリングデータ扱い）に対して推論を行う（自動ラベリングに相当）
1. 自動ラベリングの結果、評価値が閾値より高ければラベル済みデータとして扱う
1. 自動ラベリングの結果、評価値が閾値より高いデータをラベリング済みデータリストに追加。このとき自動ラベリングで作成したラベル画像をモデルの学習に使用するよう変更（あらかじめ用意したラベル画像と差し替える）
1. 評価値が高いデータの数が少なければ、未ラベリングデータの中から不足分だけ未ラベリングリストからラベリング済みリストに追加（手動ラベリング相当の処理）
1. 自動ラベリングの結果、評価値が低いものは未ラベリングリストに戻す
1. 増えたラベリング済みデータセットを使ってモデルを学習
1. 以降、未ラベリングのデータがなくなるまで繰り返す

Amazon SageMaker の Semantic Segmentation の詳細の使い方については [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/semantic-segmentation.html#semantic-segmentation-inputoutput) をご参照ください。

## Amazon SageMaker のセットアップ

In [ ]:
import boto3
from datetime import datetime
from dateutil import tz
import numpy as np
import os
import sagemaker
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role
from sagemaker.s3 import parse_s3_url

JST = tz.gettz('Asia/Tokyo')

role = get_execution_role()
print(role)
sess = sagemaker.Session()
bucket = sess.default_bucket()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
ecr_client = boto3.client('ecr', region_name=region)

prefix = 'auto-labeling-test'

### SageMaker ノートブックインスタンスの IAM ロールに権限の追加
ノートブックインスタンスで使用している IAM ロールに、以下のポリシーをアタッチしてください。

- AmazonEC2ContainerRegistryFullAccess
- AWSLambda_FullAccess

## データセットの準備
[Pascal VOC](http://host.robots.ox.ac.uk/pascal/VOC/) を使用します。 データセットの詳細は [Pascal VOC Dataset page](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/segexamples/index.html)をご参照ください。分類クラスは以下の通りです。

| Label Id |     Class     |
|:--------:|:-------------:|
|     0    |   Background  |
|     1    |   Aeroplane   |
|     2    |    Bicycle    |
|     3    |      Bird     |
|     4    |      Boat     |
|    5     |     Bottle    |
|     6    |      Bus      |
|     7    |      Car      |
|     8    |      Cat      |
|     9    |     Chair     |
|    10    |      Cow      |
|    11    |  Dining Table |
|    12    |      Dog      |
|    13    |     Horse     |
|    14    |   Motorbike   |
|    15    |     Person    |
|    16    |  Potted Plant |
|    17    |     Sheep     |
|    18    |      Sofa     |
|    19    |     Train     |
|    20    |  TV / Monitor |
|    255   | Hole / Ignore |

入力画像とラベル画像をそれぞれ data/image と data/annotation に保存し、Amazon S3 にアップロードします。また、SageMaker 学習ジョブの入力として使用するため、入力画像とラベル画像の対応を記載した manifest ファイルを作成します。

In [ ]:
%%time

print("Downloading the dataset...")
!wget -P /tmp https://fast-ai-imagelocal.s3.amazonaws.com/pascal-voc.tgz
# S3 cp may be even faster on environments where it's available:
# !aws s3 cp s3://fast-ai-imagelocal/pascal-voc.tgz /tmp/pascal-voc.tgz

print("Extracting VOC2012...")
!tar -xf /tmp/pascal-voc.tgz --wildcards pascal-voc/VOC2012*

print("Deleting /tmp files...")
!rm /tmp/pascal-voc.tgz

# Alternatively could consider using the Oxford Uni source:
#!wget -P /tmp http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
#!tar -xf /tmp/VOCtrainval_11-May-2012.tar -C pascal-voc/VOC2012
#!rm /tmp/VOCtrainval_11-May-2012.tar

print("Done!")

以下のセルでは、画像を data フォルダ以下に集約しています。

In [ ]:
import os
import shutil

# Create directory structure mimicing the s3 bucket where data is to be dumped.
VOC2012 = "pascal-voc/VOC2012"
os.makedirs("data/image", exist_ok=True)
os.makedirs("data/annotation", exist_ok=True)

# Create a list of all validation images.
# 画像1450枚くらい
with open(VOC2012 + "/ImageSets/Segmentation/val.txt") as f:
    val_list = f.read().splitlines()

# Move the jpg images in validation list to validation directory and png images to validation_annotation directory.
for i in val_list:
    shutil.copy2(VOC2012 + "/JPEGImages/" + i + ".jpg", "data/image/")
    shutil.copy2(VOC2012 + "/SegmentationClass/" + i + ".png", "data/annotation/")
    
# すべてのデータセットを使用するには以下をコメントアウトを解除してからこのセルを実行する
# # Create a list of all validation images.
# with open(VOC2012 + "/ImageSets/Segmentation/train.txt") as f:
#     val_list = f.read().splitlines()

# # Move the jpg images in validation list to validation directory and png images to validation_annotation directory.
# for i in val_list:
#     shutil.copy2(VOC2012 + "/JPEGImages/" + i + ".jpg", "data/image/")
#     shutil.copy2(VOC2012 + "/SegmentationClass/" + i + ".png", "data/annotation/")

以下のセルでは、manifest ファイルを作成しています。100MB ほどの画像データを S3 にアップロードするため実行が完了するまでに少し時間がかかります。

In [ ]:
import glob
import os

gt_job_name = 'semaseg'
train_manifest_filename = 'train.manifest'
train_file_list = glob.glob('data/image/*.jpg')
num_training_samples = len(train_file_list)

train_annotation_channel = sess.upload_data(
    path="data",
    bucket=bucket,
    key_prefix=os.path.join(prefix, "train-data"),
)

with open(train_manifest_filename, 'w') as f:
    for file in train_file_list:
        file = os.path.basename(file)
        train_data_path = os.path.join(train_annotation_channel, 'image', file)
        train_annot_data_path = os.path.join(train_annotation_channel, 'annotation', file[:-3] + 'png')
        manifest_format = '{"source-ref":"' + train_data_path + '","' + gt_job_name + '-ref":"'+ train_annot_data_path +'","'+ gt_job_name +'-ref-metadata":{"job-name":"'+ gt_job_name +'"}}'
        f.write(manifest_format + '\n')


## スクリプト実行用コンテナイメージ作成

検証用スクリプトの実行は、実行時間が長時間になることを想定してノートブックインスタンス上ではなく、Amazon SageMaker Processing を使って実現します。スクリプトが動作する環境のコンテナイメージを作成し、Amazon ECR に push します。

ECR にコンテナイメージをプッシュする権限を得るために、ノートブックインスタンスにアタッチしている IAM role に、`AmazonEC2ContainerRegistryPowerUser` policy を追加してください。

In [ ]:
!mkdir docker

In [ ]:
%%writefile docker/requirements.txt

boto3==1.20.30
mxnet==1.8.0
numpy==1.18.5
pillow==8.4.0
sagemaker==2.72.2
scikit-learn==0.24.1

In [ ]:
filepath = os.path.join( 'docker/Dockerfile')

build_config={f"""FROM python:3.7-slim-buster
    
ENV AWS_DEFAULT_REGION={region}

RUN apt update \
  && apt install -y libopenblas-dev libgomp1 \
  && apt-get clean \
  && rm -rf /var/lib/apt/lists/*
COPY requirements.txt .
RUN pip3 install -r requirements.txt
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]
"""
}

with open(filepath, 'w') as f:
    f.write('\n'.join(list(build_config)))

In [ ]:
tag = ':latest'
ecr_repository = prefix
image_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repository+tag}'

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr create-repository --repository-name $ecr_repository
 
!docker build -t $ecr_repository docker
!docker tag {ecr_repository + tag} $image_uri
!docker push $image_uri

print(f'Image has been pushed to {image_uri}.')

## Amazon SageMaker Processing でスクリプト実行

検証スクリプト `auto-labeling-test.py` を SageMaker Processing で実行します。このノートブックと同じパスに `auto-labeling-test.py` がある想定です。

In [ ]:
processing_input_dir = '/opt/ml/processing/input'
processing_output_dir = '/opt/ml/processing/output'
job_name = prefix

In [ ]:
timestamp = datetime.now(JST).strftime('%Y%m%d-%H%M%S')

EVALUATION_SCRIPT_LOCATION = "auto-labeling-test.py"
SAMPLE_PNG_LOCATION = 'pascal-voc/VOC2012/SegmentationClass/2008_003701.png'
MANIFEST_LOCATION = train_manifest_filename

sample_label_png = sess.upload_data(
    SAMPLE_PNG_LOCATION,
    bucket=bucket,
    key_prefix=os.path.join(prefix, "input", timestamp),
)
print('sample_label_png:', sample_label_png)

manifest_file = sess.upload_data(
    MANIFEST_LOCATION,
    bucket=bucket,
    key_prefix=os.path.join(prefix, "input", timestamp),
)
print('manifest_file:', manifest_file)

In [ ]:
processor = ScriptProcessor(base_job_name=job_name,
           image_uri=image_uri,
           command=['python3'],
           role=role,
           instance_count=1,
           instance_type='ml.c5.xlarge'
          )

In [ ]:
from datetime import datetime
from dateutil import tz

JST = tz.gettz('Asia/Tokyo')
timestamp = datetime.now(JST).strftime('%Y%m%d-%H%M%S')

proc_output_bucket = bucket
proc_output_prefix = job_name
project_name = 'pascal-dataset'
proc_output_s3_path = f's3://{proc_output_bucket}/{proc_output_prefix}/{project_name}/{timestamp}'

base_model_path = 's3://xxx/output/model.tar.gz'

processor.run(
            job_name='autolabel-' + project_name + '-' + timestamp,
            code=EVALUATION_SCRIPT_LOCATION, # S3 の URI でも可
             inputs=[
                     ProcessingInput(input_name='input',
                                     source=os.path.dirname(sample_label_png),
                                     destination=processing_input_dir)],
             outputs=[ProcessingOutput(output_name='output',
                                       source=processing_output_dir,
                                      destination=proc_output_s3_path)],
              arguments=[
                  '--sample-png', os.path.basename(sample_label_png),
                  '--manifest-file', os.path.basename(manifest_file),
                  '--bucket-name', proc_output_bucket,
                  '--data-output-path', proc_output_s3_path,
                  '--role-arn', role,
                  '--class-num', '21',  # ラベルの分類クラス数
                  '--background-class-id', '0',  # 背景に割り当てられたラベルID
                  '--project-name', project_name,  # 検証結果を保存する S3 パスを作成するために使用
                  '--confidence-thresh', '0.98',  # 自動ラベリング結果で最低限満たしてほしい画素単位の確信度
                  '--total-confidence-thresh', '0.85',  # confidence-threshを超える画素が何割あれば採用するかを決める閾値
                  '--num-manual-target', '1000',  # モデル再学習に必要な追加データセット数
                  '--train_ratio', '0.8',  # データセットにおける学習データの割合。残りは検証データ
                  '--timestamp', timestamp,
#                   '--base-model-path', base_model_path,  # 追加学習をする場合はベースモデルの model.tar.gz が保存されている S3 パスを指定
                  '--gt-job-name', gt_job_name,
                  '--train-instance-type', 'ml.p3.2xlarge',
                  '--input-dir', processing_input_dir,
                  '--output-dir', processing_output_dir
              ],
              wait=False
            )
print('---------------------------------')
print('Results will be saved here:', proc_output_s3_path)

from IPython.display import display, Markdown
display(Markdown(f"<a href=\"https://s3.console.aws.amazon.com/s3/buckets/{bucket}?region={region}&prefix={prefix}/{project_name}/{timestamp}/&showversions=false\" target=\"_blank\">クリックしてデータ保存場所に飛ぶ</a>"))


自動ラベリング検証ジョブが開始しました。このノートブックの構成の場合、ジョブの環境までに 2時間ほどかかります。ジョブの状況は SageMaker コンソールの左側のメニューにある[処理中]->[ジョブの処理]から確認できます。

学習ジョブの入力に使用する manifest ファイル、自動ラベリング（バッチ推論）の出力などは上記セル実行時に出力されたパスに出力されます。このパス以下にイテレーション数を示す数字のフォルダが作成され、その中にそのイテレーションで生成されたファイルが保存されます。また以下のパス直下にある png フォルダの中には、自動ラベリングによって作成されたラベル画像が保存されます。

```bash
root 
|-0/
  |-autolabel: バッチ推論の入出力ファイル
  |-train: 学習ジョブの入出力ファイル（学習済みモデルのパスは学習ジョブが生成したパスに保存される）
  |-updated-list: イテレーション官僚時点でのラベルあり画像となし画像のリスト
|-1/
|-2/
|-png/: 自動ラベリングによって作成された PNG 画像
|-report.txt: 何枚自動ラベリングされたかなどが記載されたレポート

```

## リソースの削除
このノートブックで作成したリソースを削除します。他に、このノートブックを実行したノートブックインスタンスやデータを保存した Amazon S3 バケットも不要であれば削除してください。

### ECR リポジトリの削除

In [ ]:
container_image_list = [
    ecr_repository
]
for i in container_image_list:
    try:
        ecr_client.delete_repository(
            repositoryName=i,
            force=True
        )
        print('Delete:', i)
    except Exception as e:
        print(e)
        pass